In [5]:
from requests_html import HTMLSession

session = HTMLSession()
portatil_links = []

for i in range(1,3):
    print(f'Pagina {i}')
    r = session.get(url=f"https://www.coolmod.com/portatiles-portatiles/?pagina=1{i}")
    
    articles = r.html.find('article')
    links = [article.find('a', first=True).attrs['href'] for article in articles]
    portatil_links.extend(links)

portatil_links

Pagina 1
Pagina 2


['/lenovo-v15-g2-ijl-n4500-8gb-256gb-15-6/',
 '/asus-zenbook-duo-ux8406ma-pz271w-u9-185h-32gb-2tb-14-w11h/',
 '/lenovo-v15-i5-1235u-8gb-256gb-w11h-15-6-fhd/',
 '/asus-e1504fa-nj158w-amd-r5-7520u-8gb-512-w11h-15-6/',
 '/msi-creator-pro-m16-hx-c14vjg-211es-i7-14700hx-rtx-2000-32gb-1tb-16-w11h/',
 '/medion-erazer-major-x20-md62617-i9-14900hx-rtx-4070-32gb-2tb-16-w11h/',
 '/hp-15s-fq4056ns-i7-1195g7-8gb-512gb-15-6/',
 '/medion-erazer-akoya-e16433-i7-1255u-16gb-1tb-16-w11h/',
 '/medion-erazer-crawler-e50-md62593-i5-12450h-rtx-4050-16gb-512gb-15-6-freedos/',
 '/msi-thin-15-b12ve-1490xes-i5-12450h-rtx-4050-16gb-512gb-15-6-freedos/',
 '/apple-macbook-pro-m3-8gb-1tb-14-plata/',
 '/msi-pulse-16-ai-c1vgkg-064xes-u7-155h-rtx-4070-32gb-1tb-16-freedos/',
 '/asus-f1500ea-ej2383w-i3-1115g4-8gb-512gb-w11h-15-6/',
 '/asus-zenbook-15-oled-um3504da-ma371w-r5-7535u-16gb-512gb-15-6-w11h/',
 '/portatil-asus-vivobook-15-f1504za-nj694w-intel-core-i5-1235u-8gb-512gb-ssd-15-6-win11/',
 '/medion-erazer-crawler-e3

In [6]:
portatil_links[0]

'/lenovo-v15-g2-ijl-n4500-8gb-256gb-15-6/'

In [11]:
r = session.get(url=f'https://www.coolmod.com{portatil_links[0]}')
specs = r.html.find('.collapse-content.prose', first=True)

features = {}

# Cada caracteristica del portatil
elements = specs.find('p, ul')

current_section = None
for element in elements:
    if element.tag == 'p':
        # Extract section title
        current_section = element.text.strip()
        features[current_section] = []
    elif element.tag == 'ul' and current_section:
        # Process list items
        for li in element.find('li'):
            text = li.text.strip()
            if ':' in text:
                key, value = text.split(':', 1)
                features[current_section].append({key.strip(): value.strip()})
            else:
                features[current_section].append(text)

print(features)

{}


In [ ]:
import json
from requests_html import HTMLSession

session = HTMLSession()
info_list = []

for link in portatil_links:
    r = session.get(f'https://www.mediamarkt.es{link}')
    tables = r.html.find('td')
    
    info = {}
    for i in range(0, len(tables), 2):
        clave = tables[i].text.strip()
        valor = tables[i+1].text.strip()
        info[clave] = valor
    
    info_list.append(info)

with open('info_list.json', 'w', encoding='utf-8') as f:
    json.dump(info_list, f, ensure_ascii=False, indent=4)

for info in info_list:
    print(info)